In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

from PIL import Image
from inference import Inference
from nltk.stem import PorterStemmer

/Users/mattevanoff/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/mattevanoff/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/mattevanoff/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/mattevanoff/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_path = '../../data/processed/'

In [30]:
ls ../../data/preprocessed/

example.txt           meta_json.csv         word_level_test.csv
meta.csv              meta_json.json        word_level_train.csv
meta.json             word_level_meta.csv


In [3]:
X_train = np.load(data_path + 'X_wikitext-2-train.npy')
X_test = np.load(data_path + 'X_wikitext-2-test.npy')

y_train = np.load(data_path + 'y_wikitext-2-train.npy')
y_test = np.load(data_path + 'y_wikitext-2-test.npy')

In [31]:
# import dill as pickle
# def unpickle(filename):
#     """Unpickle file"""
#     with open(filename, 'rb') as f:
#         return pickle.load(f)
# word_path_mapping = unpickle(data_path + 'word_path_mapping.pkl')
# word_path_mapping

In [4]:
word_level_df_test = pd.read_csv('../../data/preprocessed/word_level_test.csv')
word_level_df_test.head()
# Image.open(word_level_df_test.loc[0].image_path)
# available_image_letters = word_level_df.token.values.tolist()

,index,filename,folder,document,token,image_name,pos_tag,image_path
0,1111,b01-027,b01,b01-027,of,b01-027-00-05,INO,../../data/raw/word_level/b01/b01-027/b01-027-...
1,68,p03-185,p03,p03-185,.,p03-185-08-06,.,../../data/raw/word_level/p03/p03-185/p03-185-...
2,1043,h01-024,h01,h01-024,as,h01-024-02-01,IN,../../data/raw/word_level/h01/h01-024/h01-024-...
3,1162,h07-060b,h07,h07-060,the,h07-060b-03-03,ATI,../../data/raw/word_level/h07/h07-060b/h07-060...
4,1168,g06-050d,g06,g06-050,strong,g06-050d-04-02,JJ,../../data/raw/word_level/g06/g06-050d/g06-050...


In [47]:
img_path = np.random.choice(word_level_df_test.loc[word_level_df_test.token == 'work', 'image_path'].values.tolist())

'../../data/raw/word_level/j06/j06-051/j06-051-10-05.png'

In [5]:
inference_model = Inference(img_width=128, img_height=64, device='cpu')

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [30]:
len(X_test), len(X_train)

(77567, 78320)

In [6]:

import time

subset = 10000

stemmer = PorterStemmer()

y_preds = []
y_true = [y[0].lower() for y in y_test[:subset]]
N = 0
n_correct = 0
n_correct_stem = 0
bad_lines = 0


len_test = len(X_test[:subset])
start = time.time()
for i, (X, y) in enumerate(zip(X_test[:subset], y_true)):
    try:
        X = re.sub(r'<<([^;]*)>>', '[]', ' '.join(X))
    #     print(y)
        img_path_df = word_level_df_test.loc[word_level_df_test.token == y, 'image_path']
        if len(img_path_df) > 0:
            N += 1
            img_path = np.random.choice(img_path_df.values.tolist())
            # still getting 'unk'
            y_pred = inference_model.predict(X, img_path)
            if y_pred.lower() == y:
                n_correct += 1
            if stemmer.stem(y_pred) == stemmer.stem(y):
                n_correct_stem += 1

        else:
            bad_lines += 1
        if i % 100 == 0:
            print('processed {} %'.format(i / len_test))
    except:
        print('bad line')

finish = time.time()
print('Raw correct', n_correct / N)
print('Stem correct', n_correct_stem / N)
print('Time to evaluate {} sample: {}'.format(subset, finish - start))

processed 0.0 %
processed 0.01 %
processed 0.02 %
processed 0.03 %
processed 0.04 %
processed 0.05 %
processed 0.06 %
processed 0.07 %
processed 0.08 %
processed 0.09 %
processed 0.1 %
processed 0.11 %
processed 0.12 %
processed 0.13 %
processed 0.14 %
processed 0.15 %
processed 0.16 %
processed 0.17 %
processed 0.18 %
processed 0.19 %
processed 0.2 %
processed 0.21 %
processed 0.22 %
processed 0.23 %
processed 0.24 %
processed 0.25 %
processed 0.26 %
processed 0.27 %
processed 0.28 %
processed 0.29 %
processed 0.3 %
processed 0.31 %
processed 0.32 %
processed 0.33 %
processed 0.34 %
processed 0.35 %
processed 0.36 %
processed 0.37 %
processed 0.38 %
processed 0.39 %
processed 0.4 %
processed 0.41 %
processed 0.42 %
processed 0.43 %
processed 0.44 %
processed 0.45 %
processed 0.46 %
processed 0.47 %
processed 0.48 %
processed 0.49 %
processed 0.5 %
processed 0.51 %
processed 0.52 %
processed 0.53 %
processed 0.54 %
processed 0.55 %
processed 0.56 %
processed 0.57 %
processed 0.58 %
pro

In [39]:
# Raw correct 0.3185348446683459
# Stem correct 0.3212636439966415

426

In [35]:
n_correct_stem / N

0.32220513946848234

In [26]:
import re 
sample = X_test[0]
print(re.sub(r'<<([^;]*)>>', '[]', ' '.join(sample)))
print(y_true[0])

The game began development in 2010 , carrying over a large portion of the [] done on Valkyria Chronicles II .
work


In [19]:
left_text = 'the dog ran'
right_text = 'the house'

sentence = left_text + ' [] ' + right_text

sentence

'the dog ran [] the house'

In [21]:
bad_lines

0

In [ ]:
import numpy as np
import pandas as pd
import re
import time
from inference import Inference
from nltk.stem import PorterStemmer


data_path = '../../data/processed/'

X_train = np.load(data_path + 'X_wikitext-2-train.npy')
X_test = np.load(data_path + 'X_wikitext-2-test.npy')

y_train = np.load(data_path + 'y_wikitext-2-train.npy')
y_test = np.load(data_path + 'y_wikitext-2-test.npy')

word_level_df_test = pd.read_csv('../../data/preprocessed/word_level_test.csv')

# init class 
inference_model = Inference(img_width=128, img_height=64, device='cpu')


stemmer = PorterStemmer()

def evaluate_model(X_test, y_test, word_level_df_test, subset=None)
    

    y_preds = []
    y_true = [y[0].lower() for y in y_test[:subset]]
    N = 0
    n_correct = 0
    n_correct_stem = 0
    bad_lines = 0


    len_test = len(X_test[:subset])
    start = time.time()
    for i, (X, y) in enumerate(zip(X_test[:subset], y_true)):
        try:
            X = re.sub(r'<<([^;]*)>>', '[]', ' '.join(X))
        #     print(y)
            img_path_df = word_level_df_test.loc[word_level_df_test.token == y, 'image_path']
            if len(img_path_df) > 0:
                N += 1
                img_path = np.random.choice(img_path_df.values.tolist())
                # still getting 'unk'
                y_pred = inference_model.predict(X, img_path)
                if y_pred.lower() == y:
                    n_correct += 1
                if stemmer.stem(y_pred) == stemmer.stem(y):
                    n_correct_stem += 1

            else:
                bad_lines += 1
            if i % 100 == 0:
                print('processed {} %'.format((i / len_test)*100.0))
        except:
            print('bad line')

    finish = time.time()
    raw_correct = n_correct / N
    stem_correct = n_correct_stem / N
    
    print('Raw correct', raw_correct)
    print('Stem correct', stem_correct)
    print('Time to evaluate {} sample: {}'.format(subset, finish - start))
    
    return raw_correct, stem_correct


if __name__ == '__main__':
    data_path = '../../data/processed/'
#     X_train = np.load(data_path + 'X_wikitext-2-train.npy')
    X_test = np.load(data_path + 'X_wikitext-2-test.npy')
#     y_train = np.load(data_path + 'y_wikitext-2-train.npy')
    y_test = np.load(data_path + 'y_wikitext-2-test.npy')
    word_level_df_test = pd.read_csv('../../data/preprocessed/word_level_test.csv')

    # init class 
    inference_model = Inference(img_width=128, img_height=64, device='cpu')
    # evaluate model 
#     raw_correct, stem_correct = evaluate_model(X_test, y_test, word_level_df_test, subset=1000)

In [11]:
import pandas as pd

word_level_meta = pd.read_csv('../../data/preprocessed/word_level_meta.csv')
all_tokens = list(set(word_level_meta.token.values))
all_tokens = [token.lower() for token in all_tokens]

In [4]:
import numpy as np

embeddings = {}

with open('glove.6B.100d.txt') as f:
    for line in f:
        line = line.split()
        word, vector = line[0], line[1:]
        embeddings[word] = np.asarray(vector, np.float32)

In [14]:
embedding_dim = 100

word_vectors = np.zeros((len(all_tokens), embedding_dim))

for i, word in enumerate(all_tokens):
    vector = embeddings.get(word)
    if vector is not None:
        word_vectors[i, :] = vector

In [ ]:
def create_embeddings(file_path='glove.6B.100d.txt'):
    embeddings = {}

    with open(file_path) as f:
        for line in f:
            line = line.split()
            word, vector = line[0], line[1:]
            embeddings[word] = np.asarray(vector, np.float32)
    return embeddings

embeddings = create_embeddings()

In [42]:
from sklearn.metrics import pairwise_distances
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer

lemma = WordNetLemmatizer()
print(pairwise_distances([embeddings['dog']], [embeddings['broom']])[0])

# print(lemma.lemmatize('cat')
# print(pairwise_distances([embeddings['cat']], [embeddings['kitten']])[0])

converge_counter = 0
try:
    print(pairwise_distances([embeddings['fjhfhs']], [embeddings['zebra']])[0])
    converge_counter += 1
except:
    pass

[5.901278]


In [ ]:
import nltk
nltk.download('wordnet')

In [3]:
import numpy as np
import pandas as pd
import re
import time
from inference import Inference
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.metrics import pairwise_distances


def create_embeddings(file_path='glove.6B.100d.txt'):
    embeddings = {}

    with open(file_path) as f:
        for line in f:
            line = line.split()
            word, vector = line[0], line[1:]
            embeddings[word] = np.asarray(vector, np.float32)
    return embeddings



def evaluate_model(X_test, y_test, word_level_df_test, subset=None, embedding_file='glove.6B.100d.txt'):
    
    
    embeddings = create_embeddings(embedding_file)
    stemmer = PorterStemmer()
    lemma = WordNetLemmatizer()
    
    y_preds = []
    y_true = [y[0].lower() for y in y_test[:subset]]
    N = 0
    n_correct = 0
    n_correct_stem = 0
    total_similarity = 0
    bad_lines = 0

    
    len_test = len(X_test[:subset])
    start = time.time()
    for i, (X, y) in enumerate(zip(X_test[:subset], y_true)):
        try:
            X = re.sub(r'<<([^;]*)>>', '[]', ' '.join(X))
        #     print(y)
            img_path_df = word_level_df_test.loc[word_level_df_test.token == y, 'image_path']
            if len(img_path_df) > 0:
                N += 1
                img_path = np.random.choice(img_path_df.values.tolist())
                # still getting 'unk'
                y_pred = inference_model.predict(X, img_path)
                # exact match accuracy
                if y_pred.lower() == y:
                    n_correct += 1
                # stemmed accuracy 
                if stemmer.stem(y_pred) == stemmer.stem(y):
                    n_correct_stem += 1
                converge_counter = 0
                try:
                    lemma_pred = lemma.lemmatize(y_pred)
                    lemma_true = lemma.lemmatize(y)
                    similarity = pairwise_distances([embeddings[lemma_pred]], [embeddings[lemma_true]])[0][0]
                    total_similarity += similarity
                    converge_counter += 1
                except:
                    # add a score of 10 if obsure word is present
                    total_similarity += 10
                
            else:
                bad_lines += 1
            if i % 100 == 0:
                print('processed {} %'.format((i / len_test)*100.0))
        except:
            print('bad line')

    finish = time.time()
    raw_correct = n_correct / N
    stem_correct = n_correct_stem / N
    average_total_similarity = total_similarity / N
    
    
    print('Raw correct', raw_correct)
    print('Stem correct', stem_correct)
    print('Average Cosine Similarity {}. Coverage: {}'.format(average_total_similarity, round(converge_counter / N, 3)))
    
    print('Time to evaluate {} sample: {}'.format(subset, finish - start))
    print('Number of bad lines', bad_lines)
    
    return raw_correct, stem_correct, average_total_similarity


if __name__ == '__main__':

    data_path = '../../data/processed/'
#     X_train = np.load(data_path + 'X_wikitext-2-train.npy')
    X_test = np.load(data_path + 'X_wikitext-2-test.npy')
#     y_train = np.load(data_path + 'y_wikitext-2-train.npy')
    y_test = np.load(data_path + 'y_wikitext-2-test.npy')
    word_level_df_test = pd.read_csv('../../data/preprocessed/word_level_test.csv')

    # init class 
    inference_model = Inference(img_width=128, img_height=64, device='cpu')
    # evaluate model 
    raw_correct, stem_correct, average_total_similarity = evaluate_model(X_test, y_test, word_level_df_test, subset=10000)
    inference_model.sess.close()

processed 0.0 %
processed 1.0 %
processed 2.0 %
processed 3.0 %
processed 4.0 %
processed 5.0 %
processed 6.0 %
processed 7.000000000000001 %
processed 8.0 %
processed 9.0 %
processed 10.0 %
processed 11.0 %
processed 12.0 %
processed 13.0 %
processed 14.000000000000002 %
processed 15.0 %
processed 16.0 %
processed 17.0 %
processed 18.0 %
processed 19.0 %
processed 20.0 %
processed 21.0 %
processed 22.0 %
processed 23.0 %
processed 24.0 %
processed 25.0 %
processed 26.0 %
processed 27.0 %
processed 28.000000000000004 %
processed 28.999999999999996 %
processed 30.0 %
processed 31.0 %
processed 32.0 %
processed 33.0 %
processed 34.0 %
processed 35.0 %
processed 36.0 %
processed 37.0 %
processed 38.0 %
processed 39.0 %
processed 40.0 %
processed 41.0 %
processed 42.0 %
processed 43.0 %
processed 44.0 %
processed 45.0 %
processed 46.0 %
processed 47.0 %
processed 48.0 %
processed 49.0 %
processed 50.0 %
processed 51.0 %
processed 52.0 %
processed 53.0 %
processed 54.0 %
processed 55.000000

In [2]:
len(X_test)

78371

In [10]:
lemma = WordNetLemmatizer()

embeddings[lemma.lemmatize('around')]

array([ 0.049432,  0.29907 ,  0.070318,  0.085277, -0.17125 ,  0.93128 ,
       -0.67008 ,  0.54096 , -0.38016 , -0.49383 ,  0.30008 ,  0.28522 ,
        0.7559  , -0.065383,  0.56754 , -0.83206 ,  0.1329  ,  0.11873 ,
       -0.63604 , -0.29106 ,  1.0898  ,  0.65408 ,  0.58158 , -0.022854,
        0.24991 , -0.551   , -0.14216 , -0.065637,  0.10867 , -0.17114 ,
       -0.27218 , -0.20419 ,  0.9035  , -0.22882 , -0.12385 ,  0.41479 ,
       -0.15903 ,  0.23308 , -0.08571 ,  0.063337, -0.40937 , -0.79099 ,
       -0.28885 , -0.28497 , -0.2298  , -0.26469 , -0.009687, -0.066363,
        0.017328, -0.61434 ,  0.10515 ,  0.23692 , -0.069444,  1.6105  ,
       -0.61603 , -2.6838  , -0.7     , -0.31433 ,  1.5742  ,  0.81873 ,
        0.36615 ,  2.0177  , -0.30481 ,  0.30421 ,  0.077525,  0.26503 ,
        0.33716 , -0.12614 ,  0.672   ,  0.11275 ,  0.30333 , -0.32643 ,
       -0.27705 , -0.28524 ,  0.10183 , -0.4819  ,  0.76543 ,  0.13401 ,
       -0.3008  ,  0.25959 ,  0.68685 ,  0.14341 , 